In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 41.0.7 which is incompatible.


In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

/tmp/ipykernel_50/1400239515.py:17: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

In [4]:
ids = [f'0{x}' if x < 10 else str(x) for x in range(1, 46) if x != 12]  # sample 12 is missing

In [6]:
files = [TEMPDIR / "CuiHacohen2023/samples20_temp_annotated.h5ad", TEMPDIR / "CuiHacohen2023/samples21_temp_annotated.h5ad"]

In [7]:
adatas = []
for f in tqdm(files):
    adata = sc.read(f)
    adatas.append(adata)

  0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [17]:
adata = sc.concat(adatas, axis=0)

In [18]:
# Obs
adata.obs.rename({
    'nCount_RNA': 'ncounts', 
    'nFeature_RNA': 'ngenes',
    'nCount_HTO': 'ncounts_tags',
    'filename_prefix': 'sample',
    'processing_batch': 'batch',
    'biological_replicate_number': 'bio_replicate',
    'cytokine': 'perturbation'
}, axis=1, inplace=True)
adata.obs.drop(['nFeature_HTO'], axis=1, inplace=True)
adata.obs.perturbation = adata.obs.perturbation.astype(str)
adata.obs['perturbation'][pd.isna(adata.obs['perturbation'])] = 'control'
adata.obs = adata.obs[['perturbation', 'batch', 'bio_replicate', 'sample', 'ncounts', 'ngenes', 'ncounts_tags', 'hashtag_ID']]
adata.obs['nperts'] = [1-p.count('control') if type(p)==str else 0 for p in adata.obs.perturbation]
adata.obs['perturbation_type'] = 'cytokines'
adata.obs['disease'] = "healthy"
adata.obs['cancer'] = False
adata.obs['tissue_type']="primary"
adata.obs["celltype"] = 'mixed cells from draining lymph nodes'
adata.obs['organism'] = 'mouse'

/tmp/ipykernel_50/606385222.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['perturbation'][pd.isna(adata.obs['perturbation'])] = 'control'


In [19]:
adata.obs

,perturbation,batch,bio_replicate,sample,ncounts,ngenes,ncounts_tags,hashtag_ID,nperts,perturbation_type,disease,cancer,tissue_type,celltype,organism
AAACCCAAGCCAGACA-20,PBS,7.0,rep8,cytokine-samples20,125,110,300,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCAAGCCTCAAT-20,IL-17A,7.0,rep1,cytokine-samples20,245,194,388,1,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCAAGTTTCAGC-20,IL-17C,7.0,rep1,cytokine-samples20,103,93,683,3,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCACACCAAATC-20,PBS,7.0,rep8,cytokine-samples20,144,119,229,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCAGTACTAACC-20,PBS,7.0,rep8,cytokine-samples20,134,101,232,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGAGGGCCTCT-21,PBS,7.0,rep7,cytokine-samples21,105,95,408,6,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
TTTGTTGCAATCCTTT-21,C3a,7.0,rep2,cytokine-samples21,505,306,360,1,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
TTTGTTGCAATTCGTG-21,C3a,7.0,rep2,cytokine-samples21,114,96,283,1,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
TTTGTTGCAGAGGCTA-21,nan,7.0,NaN,cytokine-samples21,156,94,635,5,0,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse


In [21]:
adata = sc.read(TEMPDIR / "CuiHacohen2023.h5ad")